<a href="https://colab.research.google.com/github/Soham1729/Deep-Learning/blob/main/SohamPyne_MDS202148_Assignment_01_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Text Analytics - Assignment 1 & 2
## <center> Soham Pyne (MDS202148) <br><br>

### Importing necessary libraries

In [116]:
import gzip
import json
import re
import string
import nltk
import contractions

In [ ]:
from nltk.tokenize import sent_tokenize 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = '/content/drive/Othercomputers/My MacBook Pro/Documents/CMI-SEM 3/Text Analytics/AmazonMobileReviews/Cell_Phones_and_Accessories_5.json.gz'

### Function to read the review texts from the JSON file and appending to the list of reviews. 

In [ ]:
def read_gz_file(filename:str):
    data = []
    with gzip.open(filename, 'r') as f:
        for files in f:
            try:
                text = str(json.loads(files.decode('utf-8'))['reviewText'])
                data.append(text)    
            except KeyError:
                pass
        return data

reviews = read_gz_file(file)


In [ ]:
len(reviews)

1127672

### Writing to 'text_corpus.txt'

In [ ]:
with open('text_corpus.txt', 'w') as f:
    for review in reviews:
        f.write(f"{review}\n")

In [ ]:
pattern_dot = re.compile(r'.+\.')          #RegEx expression to filter sentences ending with period.
pattern_qmark = re.compile(r'.+\?')        #RegEx expression to filter sentences ending with question mark. 
pattern_exmark = re.compile(r'.+\!')       #RegEx expression to filter sentences ending with exclamation mark.

In [ ]:
dict_of_sentences = {'qmark':0,'exclamation':0,'period':0}
sent_found = {'qmark':[],'exclamation':[],'period':[]}

In [ ]:
def punctuation_finder(tokenized_sent):
  for se in tokenized_sent:
    if pattern_dot.search(se) is not None:
      sent_found['period'].extend(pattern_dot.findall(se))
      dict_of_sentences['period'] += 1
    elif pattern_qmark.search(se) is not None:
      sent_found['qmark'].extend(pattern_qmark.findall(se))
      dict_of_sentences['qmark'] += 1
    elif pattern_exmark.search(se) is not None:
      sent_found['exclamation'].extend(pattern_exmark.findall(se))
      dict_of_sentences['exclamation'] += 1
  return sent_found

In [ ]:
for review in reviews:
  tokenized_sent = sent_tokenize(review)
  punctuation_finder(tokenized_sent)


### The dictionay to count the occurances of the sentences with the following punctuations

In [ ]:
dict_of_sentences

{'qmark': 28197, 'exclamation': 317587, 'period': 3523068}

In [96]:
# sent_found['exclamation']

### Assignment 2

### The preprocessing steps suggested for the dataset is as follows
* Removing stopwords : This is very generic preprocessing step as stop words account for a significant portion of the vocabulary frequency count and generally it does not contain any special information.
* Removing digits : We are removing the digits while adding words to the vocabulary.
* Stemming(?) : Here we have not undertaken the stemming operations 
* Casefolding : Lowercasing the words 
* Fixing contractions : Fixing the contractions. e.g. They've = They have
* Removing punctuations : We are removing the punctuations and special characters while making the vocabulary.

In [125]:
freq_dict = {}
stop_words = set(stopwords.words("english"))
# stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
count = 0
for review in reviews:
  review = contractions.fix(review)           #Fixing the contractions. e.g. They've = They have
  review = review.replace("_", "")
  review = re.sub(r'[0-9]', '', review)       #Removing the digits from the corpus
  tokenized_word = tokenizer.tokenize(review) # Removing punctuations from the corpus
  # tokenized_word = word_tokenize(review)
  count += 1
  for word in tokenized_word:
    word = word.lower()
    #word = stemmer.stem(word)
    if word in stop_words:                    # Removing the stopwords from the corpus
      continue
    if word in freq_dict and not None:
      freq_dict[word] += 1
    else:
      freq_dict[word] = 1
  if count >= 1127672:
    break

In [126]:
freq_dict = dict(sorted(freq_dict.items(), key=lambda item: item[1],reverse=True))


### The dictionary for frequency of the words

In [127]:
len(freq_dict.keys())

118339

In [89]:
list_freq_dict_values = list(freq_dict.values())
total = sum(list_freq_dict_values)

In [128]:
(1156/118339)*100

0.9768546294966157

### Pareto rule is applicable here since less than one percent of the words account for $80\%$ occurance of the words.

In [129]:
list_freq_dict_values_ratio = [i/total for i in list_freq_dict_values]
value = sum(list_freq_dict_values_ratio[:1156])
value

0.8023511638162296

#### Vocabulary of corpus

### Total number of words without truncation

In [92]:
len(freq_dict.keys())

118632

In [81]:
# list(freq_dict.keys())

In [136]:
# '_' in freq_dict.keys()

### Removing the words with frequency lower than $(k=99)$

In [131]:
relevant_occurances = [(key,value) for (key,value) in freq_dict.items() if value>=99]

In [132]:
len(relevant_occurances)

8453

In [133]:
relevant_occurances[:20]

[('phone', 707626),
 ('case', 641996),
 ('great', 329632),
 ('one', 280478),
 ('screen', 271854),
 ('good', 257064),
 ('like', 243743),
 ('would', 211493),
 ('product', 196467),
 ('well', 194075),
 ('use', 179901),
 ('love', 145685),
 ('works', 142888),
 ('get', 141151),
 ('nice', 134117),
 ('iphone', 131959),
 ('really', 130625),
 ('protector', 127554),
 ('battery', 120104),
 ('charge', 117485)]